## Look for these snips in app.py and flavor_frontend.js and follow where they go:

# Backend:

In your app.py at some point write or generate a list for the drop down items.

In [ ]:
strict_restaurant_categories = [
    "Restaurants", "American (Traditional)", "American (New)", "Italian", 
    "Mexican", "Chinese", "Japanese", "Thai", "Vietnamese", "Indian", "Korean", 
    "Mediterranean", "Greek", "French", "Spanish", "German", "Irish", "British", 
    "Seafood", "Steakhouses", "Pizza", "Burgers", "Sushi Bars", 
    "Barbeque", "Asian Fusion", "Diners", "Breakfast & Brunch", 
    "Buffets", "Comfort Food", "Soul Food", "Southern", 
    "Tex-Mex", "Cajun/Creole", "Latin American", "Middle Eastern", "African", 
    "Caribbean", "Brazilian", "Peruvian", "Cuban", "Tapas Bars", "Gastropubs",
    "Izakaya", "Ramen", "Poke", "Hot Pot", "Dim Sum", "Fondue",
    "Fish & Chips", "Bistros", "Brasseries",
    "Donairs", "Kebab", "Falafel", "Tacos", "Cheesesteaks"
]

Setup a flask server specifically for the list of dropdown items, in addition to the one you make for your query:

In [ ]:
# Setup flask api routes
@app.route('/categories', methods=['GET'])
def get_categories():
    return jsonify({
        'strict_restaurant_categories': strict_restaurant_categories
    })

# Frontend/html: 

In your front end call the route for your list to bring over your categories list and the values to a variable in an array. Create a variable for your drop down that calls to the dropdown element on your .htlm.

In [ ]:
// Fetch the list of categories from Flask API and populate the dropdown

// call the flask route for your list
fetch('/categories')
// turn it into a json
    .then(response => response.json())     
    //put the values into an array and tie it to a variable                 
    .then(data => {
        const categories = data.strict_restaurant_categories;                    
        const dropdown = document.getElementById('category-dropdown');  // <<<<<<<<<<<< call your dropdown element from your html and assign that a variable too

        // Add each category as an option to the dropdown
        categories.forEach(category => {
            const option = document.createElement('option'); // <<<<<<<<<<<<<<<   this adds them each as an option in the drop down
            option.value = category;
            option.textContent = category;  // <<<<<<<<<<<<<<this assigns the display name in the dropdown.
            dropdown.appendChild(option); 
        });
    });


 In the html the dropdown element is a select element. This is what makes it a drop down and tracks what gets clicked on:

In [ ]:
<!-- Dropdown for selecting categories -->
            <select id="category-dropdown">
            </select>      //    <<<<<<<<<<<<<<<<< you can add a line between the select's for adding text to the dropdown display  etc.

In the javascript you can use an event listener and tie it to the html element. You can listen for when the value in the dropdown gets changed by the user, set the new selection as a variable and then use it in queries. Anything that you put inside the block of the event listener will get triggered when the event listener gets triggered. 

In [ ]:
// Event listener for dropdown changes
document.getElementById('category-dropdown').addEventListener('change', function() {  // <<<<<<<<<<<< listen for the dropdown to change
    const selectedCategory = this.value; // <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<capture the value it changed to
    function updatePieChart(data, selectedCity, selectedCategory) {   //<<<<<<<<<<<<< add anything else that you want to happen when the dropdown selection is changed and keep it inside the event listener code block.
        // Get the state of the selected city
        const cityState = data.find(r => r.city === selectedCity).state;
        
        // Get all restaurants in the selected city
        const cityRestaurants = data.filter(r => r.city === selectedCity);
    
        // Count all categories in the city
        const categoryCount = {};
        cityRestaurants.forEach(restaurant => {
            const categories = restaurant.categories.split(', ');
            categories.forEach(category => {
                if (categoryCount[category]) {
                    categoryCount[category] += 1;
                } else {
                    categoryCount[category] = 1;
                }
            });
        });
        
        
 //// etc, etc, etc

When you fetch your query route you can use that returned value as your query, since it matched the values back in your database in your mongo/postgres/sql etc. 

In [ ]:
// Fetch the restaurants for the selected category
fetch(`/query_restaurants/${selectedCategory}`)    ///<<<<<<<<<<  variable for that returned value
.then(response => response.json())
.then(data => {
    // Process the data and prepare it for the chart
    const cityCount = {};
    const locations = [];

    // Count the number of restaurants per city and collect lat/lng data
    data.forEach(restaurant => {
        const city = restaurant.city;
        const lat = parseFloat(restaurant.latitude);  // Convert to number
        const lng = parseFloat(restaurant.longitude);

        // Push the coordinates to the locations array for the heatmap
        if (lat && lng && !isNaN(lat) && !isNaN(lng)) {
            locations.push([lat, lng, 1]);  // Add intensity value
            console.log(`Added location: ${lat}, ${lng}`);  // Debug log

            ///  rest of your query result commands etc, etc, etc 

To make sure that changing the selection resets everything and repopulates anything you were making based off of it, make sure to add lines to wipe out any other elements that have been populated so that they can be repopulated. 


In our case, selecting the category is what sets off the first query and the initial generation of everything, so we were able to write 99% of everything inside the block for the dropdown event listener so that it always re-ran the chart creation and population each time. Putting this at the end allows the charts to clear and be re-populated.

In [ ]:
// Delete any existing charts so they wull refresh on reselection
if (window.myChart1 instanceof Chart) {
    window.myChart1.destroy();
}
if (window.myChart2 instanceof Chart) {
    window.myChart2.destroy();
}

## The pie chart regenerating from clicking the city in the bar chart

This one is easier. There’s a ton of cool options you can add to the charts and graphs including an option for what to do when an element is clicked. We set it up to grab the city and and pass it back through as the variable for when the pie chart is made.

In [ ]:
window.myChart1 = new Chart(ctx1, {
    type: 'bar',    //<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< On the bar chart options
    data: {
        labels: cities,
        datasets: [{
            label: `# of ${selectedCategory} restaurants`,
            data: counts,
            backgroundColor: 'rgba(98, 182, 239, 0.8)',
            borderColor: 'rgba(98, 182, 239, 1)',
            borderWidth: 1
        }]
    },
    options: {
        responsive: true,
        maintainAspectRatio: true,
        aspectRatio: 2,
        indexAxis: 'y',
        plugins: {
            legend: {
                labels: {
                    font: {
                        family: "'Helvetica Neue', 'Arial', sans-serif"
                    }
                }
            }
        },
        scales: {
            y: {
                beginAtZero: true,
                grid: {
                    display: false
                }
            }
        },
        onClick: (event, elements) => {             //<<<<<<<<<<<<<<<<<<<<<<<<<<<<down here. On click, grab the city that was clicked. Store it as the variable for making the pie chart, then update the pie chart to regenerate it.
            if (elements.length > 0) {
                const index = elements[0].index;
                const selectedCity = cities[index];
                updatePieChart(data, selectedCity, selectedCategory);
                }
            }
    }
});


Then we add the function for updatePieChart right after the event listener for the dropdown menu. By duplicating this as a function before the query fetch it allows us to call it and run from that point in the script using data that came from the bar chart and not the query. By making generating the chart an if statement that relies on it already being filled, we isolate that instance from running with the first selection of a category that would occur PRIOR to the query being submitted to grab the data.

In [ ]:
// Event listener for dropdown changes
document.getElementById('category-dropdown').addEventListener('change', function() {
    const selectedCategory = this.value;
    function updatePieChart(data, selectedCity, selectedCategory) {
        // Get the state of the selected city
        const cityState = data.find(r => r.city === selectedCity).state;
        
        // Get all restaurants in the selected city
        const cityRestaurants = data.filter(r => r.city === selectedCity);
    
        // Count all categories in the city
        const categoryCount = {};
        cityRestaurants.forEach(restaurant => {
            const categories = restaurant.categories.split(', ');
            categories.forEach(category => {
                if (categoryCount[category]) {
                    categoryCount[category] += 1;
                } else {
                    categoryCount[category] = 1;
                }
            });
        });
    
        // Get the top 5 categories
        const top5Categories = Object.entries(categoryCount)
            .sort((a, b) => b[1] - a[1])
            .slice(0, 5);
        const topCategories = top5Categories.map(item => item[0]);
        const categoryCounts = top5Categories.map(item => item[1]);
    
        // Destroy old pie chart if it exists so it recreates on city click
        if (window.myChart2 instanceof Chart) {
            window.myChart2.destroy();
        }
    
        // Create new pie chart
        const ctx2 = document.getElementById('pieChart').getContext('2d');
        window.myChart2 = new Chart(ctx2, {
            type: 'pie',
            data: {
                labels: topCategories,
                datasets: [{
                    data: categoryCounts,
                    backgroundColor: [
                        'rgba(255, 99, 132, 0.8)',
                        'rgba(54, 162, 235, 0.8)',
                        'rgba(255, 206, 86, 0.8)',
                        'rgba(75, 192, 192, 0.8)',
                        'rgba(153, 102, 255, 0.8)'
                    ],

        /// rest of the script.